## Rentabilidades de los Fondos de Inversión Colectiva (FIC)

La rentabilidad de los Fondos de Inversión Colectiva (en adelante FIC) y Fondos de Capital Privado (en adelante FCP) está determinada por la variación del valor de unidad diaria de las unidades de inversión llamadas "Participaciones".

**URL Datos Abiertos:** https://www.datos.gov.co/Hacienda-y-Cr-dito-P-blico/Rentabilidades-de-los-Fondos-de-Inversi-n-Colectiv/qhpu-8ixx


**url_base:** https://www.datos.gov.co/resource/qhpu-8ixx.json

## Configuración Google Drive - Google Sheet


1. Crear un nuevo proyecto en console.cloud.google.com
2. Habilitar API de Google Driver.
3. Creamos credenciales para acceder a la API: Google Drive API, Web Server, Aplication Data y No, I'm not using then.
4. Nombramos la credencial.
5. Orogamos un rol para el acceso.
6. Copiamos el correo electrónico y lo guardamos.
7. Elegimos formato JSON para nuestra credencial.
8. Movemos el JSON al directorio de trabajo y lo renombramos a propio gusto.
9. Activamos la API de Google Sheet
10. Compartimos el archivo de Google Sheet con el correo copiado en el punto 6. 


In [1]:
# - Importamos librerias necesarias
import requests
from datetime import datetime
import pandas as pd
import gspread
import dotenv
import os

In [2]:
# - Averiguamos la fecha minima del dataset
query = 'https://www.datos.gov.co/resource/qhpu-8ixx.json?$select=min(fecha_corte)'
response = requests.get(query)
   
if response:
    data = response.json()
    minDate = data[0].get('min_fecha_corte')
    print('Request exitoso! - la fecha minima del dataset es: ', minDate)
else:
    print('Ha ocurrido un error en el Requests de la fecha minima')


Request exitoso! - la fecha minima del dataset es:  2016-01-01T00:00:00.000


In [3]:
# - Averiguamos la fecha maxima del dataset
query = 'https://www.datos.gov.co/resource/qhpu-8ixx.json?$select=max(fecha_corte)'
response = requests.get(query)
   
if response:
    data = response.json()
    maxDate = data[0].get('max_fecha_corte')
    print('Request exitoso! - la fecha maxima del dataset es: ', maxDate)
else:
    print('Ha ocurrido un error en el Requests de la fecha maxima')


Request exitoso! - la fecha maxima del dataset es:  2021-03-28T00:00:00.000


In [4]:
# - Cargamos variables de entorno con secretos
try:
    dotenv_path = dotenv.find_dotenv('storage/keys/.env')
    dotenv.load_dotenv(dotenv_path)
    print('Dotenv localizado y actualizado!')

except:
    print('No hay variables locales!')

Dotenv localizado y actualizado!


In [5]:
dotenv_path = dotenv.find_dotenv('storage/keys/.env')


In [ ]:
# - Realizamos la primera carga de datos a un df con fecha minima y maxima como parametros
url = 'https://www.datos.gov.co/resource/qhpu-8ixx.json'
filtros = f'?$where=fecha_corte between \'{minDate}\' and \'{maxDate}\'&$limit=1500000'
query = url+filtros
headers = {'X-App-Token':os.environ.get('X_APP_TOKEN')}
response = requests.get(query, headers=headers)

if response:
    data = response.json()
    df = pd.DataFrame(data)
    print('Request exitoso! - DataFrame cargado - dimensiones: ', df.shape)

else:
    print('Ha ocurrido un error en el Requests de la carga de datos')

In [ ]:
# - Cargamos la hoja de calculo de Google Sheet y consultamos sus hojas
gc = gspread.service_account(filename='storage/keys/access-key-google-drive.json')
sheet = gc.open('FIC_dataset')
print(sheet.worksheets())

In [ ]:
# - Cargamos la hoja data y enviamos el df a la hoja data con la primera carga
sh_data = sheet.worksheet('data')
sh_data.update([df.columns.values.tolist()] + df.values.tolist())

In [ ]:
#values = df.values.tolist()
#sh_data.values_append({'values': values})
#sheet.values_append('data', {'valueInputOption': 'USER_ENTERED'}, {'values': values})

In [ ]:
# - cargamos las hojas de la hoja de calculo
#sh_summary = sheet.worksheet("summary")
#sh_data = sheet.worksheet("data")

#df_sh_summary = pd.DataFrame(sh_summary.get_all_records())
#df_sh_summary

In [ ]:
#del os.environ["X_APP_TOKEN"]


In [ ]:
# - Convertimos la fecha minima y maxima a formato datetime
#dt_minDate = datetime.strptime(minDate, '%Y-%m-%dT%H:%M:%S.%f')
#dt_maxDate = datetime.strptime(maxDate, '%Y-%m-%dT%H:%M:%S.%f')
#print(dt_minDate)
#print(dt_maxDate)

Notas:

http://veekaybee.github.io/2020/02/25/secrets/

https://github.com/trallard
https://docs.microsoft.com/en-us/windows/python/web-frameworks
https://medium.com/@tonybolanyo/variables-de-entorno-con-dotenv-en-python-4697540bf7e4

https://dev.socrata.com/foundry/www.datos.gov.co/qhpu-8ixx
https://dev.socrata.com/docs/queries/
https://dev.socrata.com/consumers/getting-started.html

https://dev.socrata.com/blog/2014/1/04/data-visualization-with-python.html
